# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298595595863                   -0.85    5.4         
  2   -8.300208192965       -2.79       -1.25    1.0    105ms
  3   -8.300434989749       -3.64       -1.89    2.6    195ms
  4   -8.300461009577       -4.58       -2.74    2.5    126ms
  5   -8.300463735094       -5.56       -3.05    2.9    147ms
  6   -8.300464179834       -6.35       -3.24   10.1    247ms
  7   -8.300464422462       -6.62       -3.39    1.5    131ms
  8   -8.300464546166       -6.91       -3.54    2.8    150ms
  9   -8.300464618780       -7.14       -3.74    1.4    124ms
 10   -8.300464635487       -7.78       -3.91    1.5    124ms
 11   -8.300464643257       -8.11       -4.27    7.0    507ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67421881271                   -0.70    6.0         
  2   -16.67859297781       -2.36       -1.14    1.0    679ms
  3   -16.67920140146       -3.22       -1.87    3.2    344ms
  4   -16.67927724049       -4.12       -2.76    3.9    355ms
  5   -16.67928592208       -5.06       -3.19    5.5    449ms
  6   -16.67928618961       -6.57       -3.49    2.5    343ms
  7   -16.67928621454       -7.60       -3.93    2.0    286ms
  8   -16.67928622127       -8.17       -4.55    2.6    354ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32521107026                   -0.56    6.4         
  2   -33.33247924519       -2.14       -1.00    1.2    1.06s
  3   -33.33409380065       -2.79       -1.73    4.9    1.38s
  4   -33.33428751169       -3.71       -2.63    2.5    1.15s
  5   -33.33431212169       -4.61       -3.00    4.2    1.49s
  6   -33.33694331438       -2.58       -2.57   10.1    1.68s
  7   -33.33694334233       -7.55       -2.55    1.1    916ms
  8   -33.33694320035   +   -6.85       -2.77    1.0    922ms
  9   -33.33694340779       -6.68       -3.68    2.4    1.00s
 10   -33.33694377850       -6.43       -3.65    4.0    1.23s
 11   -33.33694378028       -8.75       -4.14    1.0    922ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298576839798                   -0.85    5.1         
  2   -8.300260605105       -2.77       -1.58    1.0    102ms
  3   -8.300224297198   +   -4.44       -2.06    2.1    137ms
  4   -8.300369954611       -3.84       -2.26    2.2    143ms
  5   -8.300464176005       -4.03       -3.57    1.0   92.2ms
  6   -8.300464499179       -6.49       -3.69    4.4    188ms
  7   -8.300464623600       -6.91       -4.10    1.2    121ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32480120118                   -0.56    7.0         
  2   -33.26803374408   +   -1.25       -1.24    1.0    853ms
  3   +17.01991557979   +    1.70       -0.21    8.5    2.32s
  4   -33.31779035000        1.70       -1.76    6.6    2.09s
  5   -33.15021559948   +   -0.78       -1.30    4.8    1.66s
  6   -32.47919631673   +   -0.17       -1.10    4.6    1.53s
  7   -33.30464302956       -0.08       -1.73    4.4    1.34s
  8   -33.33271491316       -1.55       -2.08    2.2    1.06s
  9   -33.33374298749       -2.99       -2.03    2.9    1.24s
 10   -33.33688489972       -2.50       -2.88    2.0    1.04s
 11   -33.33693685419       -4.28       -3.06    3.0    1.28s
 12   -33.33693580417   +   -5.98      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.